In [1]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{||\mathbf{x} - \mathbf{y}||}
$$

In [2]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    normalvec: array of float, shape (n,)
        Householder tranformation reflection plane' normal vector
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    outvec = np.zeros(vec.shape)
    
    outvec[0] = np.linalg.norm(vec)
    normalvec = (vec-outvec)/np.linalg.norm(vec-outvec)
    H = np.eye(normalvec.shape[0], normalvec.shape[0])-2*np.outer(normalvec, normalvec)
    return outvec, normalvec, H

In [3]:
# Test (marked)

v = np.array([1, 2, 3])
v1, _, h = householder(v)

assert_allclose(np.dot(h, v1), v, atol=1e-10)
assert_allclose(np.dot(h, v), v1, atol=1e-10)

In [4]:
rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, _, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the QR decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $QR$ decomposition of $\mathbf{A}$. 

In [5]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    r = np.array(a, copy=True, dtype=float)
    m, n = r.shape
    q = np.eye(m,m)
    
    for i in range(min(n,m)-1):
        _, _, h = householder(r[i:,i])
        eye = np.eye(m,m)
        eye[i:,i:]=h
        h = eye
        q = np.dot(q, h.T)
        r = np.dot(h, r)
    return q, r

In [6]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [7]:
# Test (marked)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [8]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here (Russian or English, up to you)*

### The results are close

### More tests

In [9]:
%%time

a = np.random.random(size=(4, 5))
q, r = qr_decomp(a)
assert_allclose(np.dot(q.T,a), r, atol=1e-10)

a = np.random.random(size=(1, 2))
q, r = qr_decomp(a)
assert_allclose(np.dot(q.T,a), r, atol=1e-10)

a = np.random.random(size=(200, 100))
q, r = qr_decomp(a)
assert_allclose(np.dot(q.T,a), r, atol=1e-10)

a = np.random.random(size=(1000, 1000))
q, r = qr_decomp(a)
assert_allclose(np.dot(q.T,a), r, atol=1e-10)

CPU times: user 6min 29s, sys: 38.9 s, total: 7min 8s
Wall time: 1min 54s


### It works fine

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. Write a second function, which uses the reflection vertices to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests.

In [10]:
def qr_decomp_adv(a):
    """Compute the QR decomposition of a matrix.

        Parameters
        ----------
        a : ndarray, shape(m, n)
            The input matrix

        Returns
        -------
        u_s : list of ndarrays, shape (m,)
            List of Householder tranformation reflection planes' normal vectors
        r : ndarray, shape(m, n)
            The upper triangular matrix
    """
    r = np.array(a, copy=True, dtype=float)
    m, n = r.shape    
    u_s = []
    for i in range(min(n,m)-1):
        _, u, _ = householder(r[i:,i])
        u = np.insert(u, 0, np.zeros(i))
        u_s.append(u)
        r -= 2 * np.dot(u.reshape((-1,1)), np.dot(u.reshape((1,-1)), r))
    return u_s, r

def Q_T_mul(a, u_s):
    """Compute the matrix product Q_T a,
        where a is an arbitrary matrix
        and Q_t is a matrix product H_n...H_1,
        where H_k is a Householder matrix specified by a reflection vector u_s[k]
        
        Parameters
        ----------
        a : ndarray, shape(m, n)
            The input matrix
        u_s : list of ndarrays, shape (m,)
              List of Householder tranformation reflection planes' normal vectors
            
        Returns
        -------
        a1 : ndarray, shape (m, n)
             Matrix product Q_T a
    """
    a1 = np.copy(a)
    for u in u_s:
        a1 -= 2 * np.dot(u.reshape((-1,1)), np.dot(u.reshape((1,-1)), a1))
    return a1

### Tests again

In [11]:
%%time

a = np.random.random(size=(4, 5))
u_s, r = qr_decomp_adv(a)
assert_allclose(Q_T_mul(a, u_s), r, atol=1e-10)

a = np.random.random(size=(1, 2))
u_s, r = qr_decomp_adv(a)
assert_allclose(Q_T_mul(a, u_s), r, atol=1e-10)

a = np.random.random(size=(200, 100))
u_s, r = qr_decomp_adv(a)
assert_allclose(Q_T_mul(a, u_s), r, atol=1e-10)

a = np.random.random(size=(1000, 1000))
u_s, r = qr_decomp_adv(a)
assert_allclose(Q_T_mul(a, u_s), r, atol=1e-10)

CPU times: user 49.3 s, sys: 28.8 s, total: 1min 18s
Wall time: 22 s


### Works faster